# Basic RNA circuit simulation 



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from bioreaction.simulation.basic_sim import convert_model

import jax
import numpy as np
import pandas as pd
import os
import sys

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-paper')
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from src.utils.data.data_format_tools.common import load_json_as_dict
from src.utils.common.setup_new import prepare_config, construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller
from tests_local.shared import five_circuits

config = load_json_as_dict('../tests_local/configs/simple_circuit.json')


In [4]:



def update_model_rates(model, a=None, d=None, ka=None, kd=None):
    for i, r in enumerate(model.reactions):
        if not r.input:  # 0 -> RNA
            if a is not None:
                model.reactions[i].forward_rate = a[model.species.index(
                    r.output[0])]
                model.reactions[i].reverse_rate = 0
        elif not r.output:  # RNA -> 0
            if d is not None:
                model.reactions[i].forward_rate = d[model.species.index(
                    r.input[0])]
                model.reactions[i].reverse_rate = 0
        else: # unbound RNA -> bound RNA
            if ka is not None:
                model.reactions[i].forward_rate = ka[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
            if kd is not None:
                model.reactions[i].reverse_rate = kd[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
    return model


def make_params(model, scale_rates=True):
    sim_model = convert_model(model)

    if scale_rates:
        m = np.max([sim_model.forward_rates.max(), sim_model.reverse_rates.max()])
    else:
        m = 1
    inputs = sim_model.inputs
    outputs = sim_model.outputs
    forward_rates = sim_model.forward_rates/m
    reverse_rates = sim_model.reverse_rates/m
    return inputs, outputs, forward_rates, reverse_rates, m

Use the 5 tester circuits :)

In [5]:
config = prepare_config(config)
config['include_prod_deg'] = True
config['simulation']['use_rate_scaling'] = False

q = 2
p = 30
config['molecular_params_factor'] = p
config['simulation']['interaction_factor'] = 1
config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * q
config['molecular_params']['creation_rate' + '_per_molecule'] = config['molecular_params']['creation_rate' + '_per_molecule'] * q
# config['simulation']['dt'] = 0.1
# config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * p
# config['molecular_params']['degradation_rate'] = config['molecular_params']['degradation_rate'] * p
circuits, config, result_writer = five_circuits(config)

xla_bridge.py:backends():355: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:  INFO
xla_bridge.py:backends():355: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Host Interpreter CUDA INFO
xla_bridge.py:backends():355: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client' INFO
xla_bridge.py:backends():355: Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this. INFO
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/0_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experi

In [6]:


def load_circuit(top_dir, circuit_name, config):
    dp = os.path.join(top_dir, 'circuits', circuit_name + '.fasta')
    interactions = {'binding_rates_association': config['molecular_params']['association_binding_rate' + '_per_molecule'],
                    'binding_rates_dissociation': os.path.join(top_dir, 'binding_rates_dissociation', circuit_name + '_' + 'binding_rates_dissociation' + '.csv'),
                    'eqconstants': os.path.join(top_dir, 'eqconstants', circuit_name + '_' + 'eqconstants' + '.csv'),
                    'energies': os.path.join(top_dir, 'energies', circuit_name + '_' + 'energies' + '.csv'),
                    'binding_sites': os.path.join(top_dir, 'binding_sites', circuit_name + '_' + 'binding_sites' + '.csv')}

    return construct_circuit_from_cfg({
        'data_path': dp,
        'interactions': interactions
    }, config)

# tester_circuit = load_circuit(top_dir='data/ensemble_mutation_effect_analysis/2023_04_04_092945/generate_species_templates',
#              circuit_name='toy_mRNA_circuit_1844', config=config)


tester_circuits = []
for name in ['toy_mRNA_circuit_21566', 'toy_mRNA_circuit_24706', 'toy_mRNA_circuit_19768', 'toy_mRNA_circuit_2983', 'toy_mRNA_circuit_22658']:
    config['mutations'] = pd.read_csv(os.path.join(
        '..', 'data/ensemble_mutation_effect_analysis/2023_04_11_192013/mutation_effect_on_interactions_signal', name, 'mutations.csv')).to_dict()
    tester_circuits.append(load_circuit(top_dir='data/ensemble_mutation_effect_analysis/2023_04_04_092945/generate_species_templates',
                                        circuit_name=name, config=config))

# config['interactions'] = {}
# config['interactions'] = None
# tester_circuit = construct_circuit_from_cfg(
#     {'data_path': '../tester.fasta'}, config)
[circuits.append(tester_circuit) for tester_circuit in tester_circuits]


[None, None, None, None, None]

In [15]:
pd.DataFrame(config['mutations'])


,mutation_name,template_name,template_seq,mutation_types,positions,count,algorithm,sequence_type,template_file
0,RNA_0_m1-0,RNA_0,AGUUGAGGCUUCGUGCACGG,[7],[4],1,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
1,RNA_0_m1-1,RNA_0,AGUUGAGGCUUCGUGCACGG,[7],[6],1,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
2,RNA_0_m1-2,RNA_0,AGUUGAGGCUUCGUGCACGG,[4],[8],1,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
3,RNA_0_m1-3,RNA_0,AGUUGAGGCUUCGUGCACGG,[8],[7],1,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
4,RNA_0_m1-4,RNA_0,AGUUGAGGCUUCGUGCACGG,[6],[19],1,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
...,...,...,...,...,...,...,...,...,...
115,RNA_2_m15-3,RNA_2,UUCGGAGCAGCGCGGACGAG,"[0, 8, 9, 2, 8, 6, 5, 0, 6, 6, 7, 7, 7, 1, 5]","[18, 4, 1, 5, 17, 11, 7, 15, 19, 3, 13, 14, 9,...",15,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
116,RNA_2_m15-4,RNA_2,UUCGGAGCAGCGCGGACGAG,"[7, 1, 4, 3, 1, 7, 5, 6, 6, 8, 1, 8, 6, 11, 5]","[19, 5, 10, 2, 8, 13, 16, 4, 11, 14, 15, 9, 6,...",15,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
117,RNA_2_m15-5,RNA_2,UUCGGAGCAGCGCGGACGAG,"[2, 8, 6, 7, 8, 1, 1, 8, 3, 4, 9, 11, 5, 5, 3]","[8, 14, 19, 13, 4, 5, 18, 11, 10, 16, 0, 1, 12...",15,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...
118,RNA_2_m15-6,RNA_2,UUCGGAGCAGCGCGGACGAG,"[5, 7, 3, 7, 4, 6, 0, 6, 7, 10, 7, 2, 7, 0, 7]","[7, 13, 10, 17, 2, 3, 5, 6, 4, 0, 14, 15, 11, ...",15,random,RNA,data/ensemble_mutation_effect_analysis/2023_04...


In [7]:

circuit_modeller = CircuitModeller(result_writer=result_writer, config=config)
circuits = circuit_modeller.batch_circuits(circuits=circuits, methods={"compute_interactions": {}})
i = -1
# for i in range(len(circuits)):
#     circuits[i].interactions.binding_rates_association = circuits[i].interactions.binding_rates_association * q
#     circuits[i].interactions.binding_rates_dissociation = circuits[i].interactions.binding_rates_dissociation * q
#     circuits[i].update_species_simulated_rates(circuits[i].interactions)


simulator.py:check_IntaRNA_path():115: Could not detect IntaRNA on system: CompletedProcess(args='which IntaRNA', returncode=1, stdout='', stderr='') WARNING
circuit_manager_new.py:batch_circuits():565: 	From 10 circuits, a total of 10 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():572: 		Starting new round of viable circuits (0 - 10 / 10) WARNING
circuit_manager_new.py:batch_circuits():590: 		Making subcircuits 0 - 10 took 6.2e-05s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 0 - 10 circuits (out of 0 - 10 (total: 10)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - 0_weak: compute_interactions WARNING
circuit_manager_new.py:run_batch():663: 		Method compute_interactions took 0.071107s WARNING
circuit_manager_new.py:batch_circuits():610: Single batch: 0:00:00.072345 
Projected time: 0.072345s 
Total time: 0:00:00.072355 WARNING


In [8]:
circuits[i].interactions.binding_rates_dissociation

array([[0.00052282, 0.22631998, 0.01330353],
       [0.22631998, 0.22631998, 0.00028829],
       [0.01330353, 0.00028829, 0.22631998]])

In [9]:
circuits[i].interactions.binding_rates_association

array([[0.00150958, 0.00150958, 0.00150958],
       [0.00150958, 0.00150958, 0.00150958],
       [0.00150958, 0.00150958, 0.00150958]])

In [10]:
config['molecular_params']['creation_rate']

141.0

In [11]:
config['molecular_params']['degradation_rate']


0.3525

In [13]:

circuits = circuit_modeller.batch_circuits(
    circuits=circuits, 
    methods={
        # "compute_interactions": {},
        "init_circuits": {'batch': True},
        'simulate_signal_batch': {'ref_circuit': None,
                                  'batch': config['simulation']['use_batch_mutations']},
        'write_results': {'no_visualisations': False, # config['experiment']['no_visualisations'],
                          'no_numerical': False} #config['experiment']['no_numerical']}
    }
)

circuit_manager_new.py:batch_circuits():565: 	From 10 circuits, a total of 10 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():572: 		Starting new round of viable circuits (0 - 10 / 10) WARNING
circuit_manager_new.py:batch_circuits():590: 		Making subcircuits 0 - 10 took 1288.774627s WARNING
circuit_manager_new.py:batch_circuits():596: 	Batching 0 - 10 circuits (out of 0 - 10 (total: 10)) (Circuits: 0 - 10 of 10) WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - 0_weak: init_circuits WARNING


Done:  0:00:03.763780


circuit_manager_new.py:run_batch():663: 		Method init_circuits took 4.440769s WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - 0_weak: simulate_signal_batch WARNING
circuit_manager_new.py:prepare_internal_funcs():495: Signal differs between circuits, but only first signal used for simulation. WARNING
circuit_manager_new.py:simulate_signal_batch():343: 		Simulating signal took 3.927567s WARNING


Done:  0:00:03.875461


circuit_manager_new.py:simulate_signal_batch():401: 		Calculating analytics took 4.384167s WARNING
circuit_manager_new.py:run_batch():663: 		Method simulate_signal_batch took 8.654215s WARNING
circuit_manager_new.py:run_batch():627: 		Running 10 Subcircuits - 0_weak: write_results WARNING
circuit_manager_new.py:run_batch():663: 		Method write_results took 6.088773s WARNING
circuit_manager_new.py:batch_circuits():610: Single batch: 0:21:47.964460 
Projected time: 1307.96446s 
Total time: 0:21:47.964473 WARNING


Log

1.
q = 5
p = 2
-> not great

2. 
q = 2
p = 5
-> good, the weak ones could be more step-like

3. 
q = 3
p = 5
-> Same-looking slope for signal, but doesn't reach as high as 2. 

4. 
q = 2
p = 10
-> better, more step-like on strong, but still a bit dragging on weaker ones

5. 
q = 2
p = 15
-> even better, more step-like, but no overshoot like you get in the no prod/deg

6. 
q = 5
p = 50
-> spiky, but now a bit too dominating with prod / deg




